In [549]:
from random import random
import numpy as np
from scipy import rand
import scipy.linalg

np.set_printoptions(precision=2, suppress=True)

Opdracht 1. Het valt ons op dat er iets mis gaat in de code, de linker onder
singuliere waarde is af en toe negatief, terwijl dit positief zou moeten zijn
in de latere opdracht hebben we algemenere svd benadering gemaakt die ook voor
een 2x2 matrix werkt.

In [550]:
def rot2d(a, b):
    factor = 1/(np.sqrt(a**2+b**2))
    matrix = np.array([[a, b], [-b, a]])
    return factor*matrix

def approx_svd(A, j):
    if j == 0:
        return A
    prevA = approx_svd(A, j-1)
    if j % 2 == 0:
        return prevA @ rot2d(prevA[0][0], prevA[0][1]).T
    return rot2d(prevA[0][0], prevA[1][0]) @ prevA

def sigma(a):
    U, s, vH = np.linalg.svd(a)
    return s

A = np.random.rand(2, 2)
print(A)
# print(approx_svd(A, 1))
# print(approx_svd(A, 2))
# print(approx_svd(A, 15))
print(approx_svd(A, 10))
print(sigma(A))

[[0.47 0.47]
 [0.81 0.02]]
[[ 0.97  0.  ]
 [-0.   -0.39]]
[0.97 0.39]


Opdracht 2.De nulrechts functie kan alle waardes in de bovendriehoek in nullen
verranderen en werkt dus alleen voor i,j met j>i. De nullinks functie kan alle
waardes in de onderdriehoek in nullen verranderen en werkt dus alleen voor i, j
met i>j.

In [551]:

def rot(n, a, b, i):
    boven = np.eye(i)
    onder = np.eye(n-i-2)
    return scipy.linalg.block_diag(boven, rot2d(a, b), onder)

def nul_links(A, i, j):
    a = A[i-2][j-1]
    b = A[i-1][j-1]
    return rot(len(A), a, b, (i-2))


def rot_rechts(n, a, b, i):
    boven = np.eye(i)
    onder = np.eye(n-i-2)
    return scipy.linalg.block_diag(boven, rot2d(a, b).T, onder)


def nul_rechts(A, i, j):
    c = A[i-1][j-2]
    d = A[i-1][j-1]
    return rot_rechts(len(A[0]), c, d, (j-2))

def links_rotaties(A):
    matrix = A
    for i in range(2, len(A)+1)[::-1]:
        matrix = nul_links(matrix, i, 1)@matrix
    for i in range(3, len(A)+1)[::-1]:
        matrix = nul_links(matrix, i, 2) @ matrix
    matrix = nul_links(matrix, 4, 3) @ matrix
    return matrix

Opdracht 3. De stappen die worden toegepast zijn te vinden in opdracht 4.
De volgorde waarin deze stappen moeten worden uitgevoerd maakt uit. Een
verkeerde volgorde resulteert erin dat een eerder gemaakte nul wordt verpest bij
het maken van een nieuwe nul in de matrix.
De volgorde werkt als volgt bij een arbitrair grote matrix. Om een
boven-bidiagonaal matrix te maken begin je met de eerste kolom van onder naar
boven in nullen te verranderen. Vervolgens verrander je de eerste rij in nullen,
beginnend bij het meest rechter ellement werkend tot 1 voor de diagonaal.
Vervolgens herhaal je dit op dezelfde volgorde voor de tweede kolom, en de
tweede rij, vervolgens voor de derde kolom en derde rij, en zo voort tot je
alle rijen langs bent gegaan.

Opdracht 4.

In [552]:
def bidiagonaliseer(A):
    matrix = A
    for i in range(2, len(A)+1)[::-1]:
        matrix = nul_links(matrix, i, 1)@matrix
    matrix = matrix @ nul_rechts(matrix, 1, 3)
    for i in range(3, len(A)+1)[::-1]:
        matrix = nul_links(matrix, i, 2) @ matrix

    matrix = nul_links(matrix, 4, 3) @ matrix

    return matrix

def links_rotaties(A):
    matrix = A
    for i in range(2, len(A)+1)[::-1]:
        matrix = nul_links(matrix, i, 1)@matrix
    for i in range(3, len(A)+1)[::-1]:
        matrix = nul_links(matrix, i, 2) @ matrix
    matrix = nul_links(matrix, 4, 3) @ matrix
    return matrix

def bidiagonaliseer_stap(A):
    U = links_rotaties(A)
    W = A @ nul_rechts(A, 1, 3)
    return (U, bidiagonaliseer(A), W)


Opdracht 5. Door de twee waardes op de bovendiagonaal in nullen te verranderen,
oftwel voor een 4x3 de rechts rotatie voor i=1 en j=2 en vervolgens nog een
rechtsrotatie voor i=2 en j=3. Andersom om van een onder-bidiagonaal een
boven-bidiagonaal matrix te maken moet tweemaal een linksrotatie worden gedaan,
eenmaal voor i=2 en j=1 en daarna nog een linksrotatie voor i=3 en j=2.

Opdracht 6. Bij het vergelijken met numpy valt ons op dat onze U en V* ongelijk
zijn aan de U en V* van numpy. Onze U is de transpose van de numpy U en net zo
is onze V* de transpose van de numpy V*. Het blijkt dat uit deze methode dus
niet U en V* komen maar $U^T en V.$

In [553]:
def boven_bidiagonaliseer_alle(A):
    N = len(A)
    M = len(A[0])
    V = np.eye(len(A[0]), len(A[0]))
    U = np.eye(len(A), len(A))
    for i in range(1, max(N, M)):
        for j in range(N, i, -1):
            if i > M:
              continue
            links=nul_links(A, j, i)
            A = links@A
            U = links@ U
        for j in range(M, i+1, -1):
            if i > N:
                continue
            rechts = nul_rechts(A, i, j)
            A = A@ rechts
            V = V @ rechts
    return A, V, U

def boven_naar_onder(A, V):
    n = len(A)
    m = len(A[0])
    i = 2
    j = 1
    while j <= n and i <= m:
        rechts =nul_rechts(A, j, i)
        A = A @ rechts
        V= V@ rechts
        i += 1
        j += 1
    return A, V


def onder_naar_boven(A, U):
    n = len(A)
    m = len(A[0])
    i = 1
    j = 2
    while j <= n and i <= m:
        links=nul_links(A, j, i)
        A = links @ A
        U = links @ U
        i += 1
        j += 1
    return A, U


def iteratie(A, n):
    A, V, U = boven_bidiagonaliseer_alle(A)
    for _ in range(n):
        A, V = boven_naar_onder(A, V)
        A, U = onder_naar_boven(A, U)
    return A, V, U


N=3
M=5
Matrix = np.random.rand(N, M)
U_np,A_np, V_np = np.linalg.svd(Matrix)

A, V, U = iteratie(Matrix, 16)

print("Onze uitkomst:")
print("U=")
print(U)
print("A=")
print(A)
print("V*=")
print(V)
print("Start matrix:")
print(Matrix)
print("U*A*V=")
print(U@A@V)
print("U*A*V=")
print(U.T@A@V.T)

print("Numpy uitkomst:")
print("U=")
print(U_np)
print("A=")
print(A_np)
print("V*=")
print(V_np)
print("Start matrix:")
print(Matrix)
print("U*A*V=")
print(U_np@A@V_np)




Onze uitkomst:
U=
[[ 0.59  0.41  0.69]
 [-0.13  0.9  -0.42]
 [-0.79  0.15  0.59]]
A=
[[ 1.68  0.    0.   -0.    0.  ]
 [ 0.    0.62 -0.    0.   -0.  ]
 [-0.    0.    0.23  0.   -0.  ]]
V*=
[[ 0.17 -0.19 -0.2  -0.95  0.  ]
 [ 0.35  0.9  -0.08 -0.1   0.21]
 [ 0.57 -0.01  0.17  0.07 -0.8 ]
 [ 0.68 -0.36 -0.39  0.28  0.42]
 [ 0.26 -0.12  0.88 -0.11  0.36]]
Start matrix:
[[0.23 0.28 0.53 0.78 0.11]
 [0.01 0.74 0.39 0.25 0.14]
 [0.22 0.16 0.68 0.83 0.46]]
U*A*V=
[[ 0.35  0.04 -0.19 -0.96 -0.07]
 [ 0.1   0.55 -0.02  0.15  0.2 ]
 [-0.12  0.34  0.28  1.26 -0.09]]
U*A*V=
[[0.23 0.28 0.53 0.78 0.11]
 [0.01 0.74 0.39 0.25 0.14]
 [0.22 0.16 0.68 0.83 0.46]]
Numpy uitkomst:
U=
[[-0.59 -0.13 -0.79]
 [-0.41  0.9   0.15]
 [-0.69 -0.42  0.59]]
A=
[1.68 0.62 0.23]
V*=
[[-0.17 -0.35 -0.57 -0.68 -0.26]
 [-0.19  0.9  -0.01 -0.36 -0.12]
 [-0.2  -0.08  0.17 -0.39  0.88]
 [-0.85  0.   -0.29  0.44  0.06]
 [ 0.42  0.23 -0.75  0.26  0.37]]
Start matrix:
[[0.23 0.28 0.53 0.78 0.11]
 [0.01 0.74 0.39 0.25 0.14]
 [0.

$\LaTeX{}$

$5x + 3 = 1$